In [1]:
import ipywidgets as wgt
import numpy as np
import pandas as pd
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.offline as py
#import plotly.figure_factory as ff
import plotly.graph_objs as go
tdf=pd.read_csv('chess2_rep_moreinfo.tab', sep='\t', index_col=0)
tdf['excov']=pd.to_numeric(tdf['excov'], errors='coerce').fillna(0)
tdf['cdcov']=pd.to_numeric(tdf['cdcov'], errors='coerce').fillna(0)
tdf['tcoding']=np.where(tdf['cdlen'].ne(0), True, False)
tdf['tnovel']=np.where(tdf['status'].isin(['novel', 'known_fantom']), True, False)
tdf['gcoding']=np.where(tdf['gtype'].eq('protein_coding'), True, False)
tdf['gnovel']=np.where(tdf['gstatus'].isin(['novel', 'known_fantom']), True, False)
tdf[(tdf['tcoding'].eq(True))].head()

,gstatus,gtype,status,type,asm,chr,strand,excount,exlen,cdlen,exreps,cdreps,excov,cdcov,tcoding,tnovel,gcoding,gnovel
id,,,,,,,,,,,,,,,,,,
CHS.6.1,known_refseq,protein_coding,known_refseq,coding,yes,chr1,+,1,918,918,.,.,0.0,0.0,True,False,True,False
CHS.9.1,known_refseq,protein_coding,known_refseq,coding,yes,chr1,+,4,744,402,Satellite|telo:62,.,8.3,0.0,True,False,True,False
CHS.9.2,known_refseq,protein_coding,known_refseq,coding,yes,chr1,+,4,762,402,Satellite|telo:80,.,10.5,0.0,True,False,True,False
CHS.9.3,known_refseq,protein_coding,known_refseq,coding,no,chr1,+,4,1014,402,"LINE|L1:163,DNA|hAT-Charlie:103",.,26.2,0.0,True,False,True,False
CHS.9.4,known_refseq,protein_coding,known_refseq,coding,yes,chr1,+,4,841,402,LINE|L1:159,.,18.9,0.0,True,False,True,False


In [2]:
seltype=wgt.SelectMultiple(
    options=sorted(tdf.type.unique()),
    value=[],
    rows=4,
    description='TYPE',
)
selstatus=wgt.SelectMultiple(
    options=sorted(tdf.status.unique()),
    value=['novel'],
    rows=4,
    description='STATUS',
)
hbox2=wgt.HBox([selstatus, seltype])
display(hbox2)
layw=wgt.Layout(width='16em')
excov=wgt.IntSlider(value=0, min=0, max=100, step=5, layout=layw)
cdcov=wgt.IntSlider(value=0, min=0, max=100, step=5, layout=layw)
excount=wgt.BoundedIntText(value=1, min=1, max=10, description='At least',
                          layout=wgt.Layout(width='11em'))
hbox=wgt.HBox([wgt.VBox([wgt.Label(value="Min. exon cov%:"), excov]),
                  wgt.VBox([wgt.Label(value="Min. CDS cov%:"), cdcov]),
              excount, wgt.Label(value="exons")])
display(hbox)

runbtn=wgt.Button(
    description='Query',
    #disabled=False,
    #button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Query',
    #icon='check'
)
rlabel=wgt.Label(value="---", layout=wgt.Layout(border='2px solid red', padding='0 12px'))
rlabelres=wgt.Label(value="---", layout=wgt.Layout(border='2px solid red', padding='0 12px'))
def btnRunClick(b):
    rlabel.value=''
    rdf=tdf
    if len(selstatus.value)==4 or len(selstatus.value)==0:
        rlabel.value='any status'
    else: 
        rlabel.value=' | '.join(selstatus.value)
        rdf=tdf[(tdf['status'].isin(selstatus.value))]
        if len(selstatus.value)==3 and not 'novel' in selstatus.value:
            rlabel.value='known'
        else:
            if len(selstatus.value)>1: rlabel.value='('+rlabel.value+')'
    stype=''
    
    if not(len(seltype.value)==4 or len(seltype.value)==0):
        stype=' | '.join(seltype.value)
        rdf=rdf[(rdf['type'].isin(seltype.value))]
        
    if rlabel.value and stype: 
        rlabel.value+=' &  '
        if len(seltype.value)>1: stype='('+stype+')'
    rlabel.value+=stype
    if excount.value>1:
        rdf=rdf[(rdf['excount']>=excount.value)]
    if excov.value>0:
       rdf=rdf[(rdf['excov']>=excov.value)]    
    if cdcov.value>0:
       rdf=rdf[(rdf['cdcov']>=cdcov.value)]

    rlabelres.value=str(len(rdf))

runbtn.on_click(btnRunClick)

hbx=wgt.HBox(children=[runbtn, rlabel, rlabelres], layout=wgt.Layout(justify_content='center'))
display(hbx)
btnRunClick(True)

In [3]:
cdf=pd.crosstab(tdf['status'],tdf['type'], margins=True)
del cdf.index.name
del cdf.columns.name
cdf

,coding,coding_maybe,noncoding,undetermined,All
known_fantom,0,19250,0,10791,30041
known_gencode,13272,0,36095,0,49367
known_refseq,126579,85,29727,1352,157743
novel,0,1807,0,84869,86676
All,139851,21142,65822,97012,323827


In [4]:
cdk=pd.crosstab(tdf['tnovel'],tdf['tcoding'], margins=True)
#del cdk.index.name
#del cdk.columns.name
cdk.index = ['Known','Novel', 'All']
cdk.columns=['No CDS', 'With CDS', 'All']
print("Novel transcripts and CDS presence (including novel isoforms)")
cdk

Novel transcripts and CDS presence (including novel isoforms)


,No CDS,With CDS,All
Known,77053,130057,207110
Novel,115311,1406,116717
All,192364,131463,323827


In [18]:
cdg=pd.crosstab(tdf['gnovel'],tdf['tcoding'], margins=True)
#cdg.index = ['In known genes','In novel genes', 'All']
#cdg.columns=['No CDS', 'With CDS', 'All']
print("Transcripts in novel genes")
cdg

Transcripts in novel genes


tcoding,False,True,All
gnovel,,,
False,187658,130057,317715
True,4706,1406,6112
All,192364,131463,323827


In [6]:
def plotCH(df, column, gtitle, xlabel, ylabel='transcripts'):
    hist_freqs, bins = np.histogram(a=df[column], bins=20)
    hist_freqs=np.flip(hist_freqs)
    hist_cfreqs=np.cumsum(hist_freqs)
    hist_cfreqs=np.flip(hist_cfreqs)
    data=[ go.Bar(x=bins[1:], y=hist_cfreqs[1:], text=hist_cfreqs[1:], textposition='outside') ]
    layout=go.Layout(title=str(df.shape[0])+gtitle, 
                 xaxis=dict(tick0=0, dtick=5, title=xlabel, 
                            tickangle=-45), yaxis=dict(title=ylabel))
    fig=go.Figure(data=data, layout=layout)
    py.iplot(fig, filename='bar chart')

In [7]:
ndf=tdf[(tdf['tnovel'].eq(True))]
plotCH(ndf, 'excov', ' novel transcripts - repeat content', 
                 'Min. % exon coverage by repeats')

In [15]:
pdf=ndf[(ndf['tcoding'].eq(True))]
pdf=pdf[(pdf['gnovel'].eq(True))]
plotCH(pdf, 'excov', ' coding transcripts in novel genes', 'Min. % exon coverage by repeats')
plotCH(pdf, 'cdcov', ' coding transcripts in novel genes', 'Min. % CDS coverage by repeats')

In [17]:
mxdf=pdf[(pdf['excount']>1)]
plotCH(mxdf, 'cdcov', ' multi-exon coding transcripts in novel genes', 'Min. % CDS coverage by repeats')

In [22]:
kdf=tdf[(tdf['tcoding'].eq(True))]
rdf=kdf[(kdf['status'].eq('known_gencode'))]
rdf=rdf[(rdf['excount']>1)]
plotCH(rdf, 'cdcov', ' multi-exon coding transcripts in Gencode exclusive genes', 'Min. % CDS coverage by repeats')
